In [1]:
from pymongo import MongoClient
import pandas as pd
import requests

In [2]:
client = MongoClient("mongodb://localhost/projectdb")
db = client.get_database()

In [3]:
airport = list(db.airports.find({}))

In [4]:
#Searching the airport with more companies in 30 km
companies_number_air=[]
companies_geolocation_air=[]
for e in airport:
    query = {
        "geolocation": {
            "$near": {
            "$geometry": e["geolocation"],
            "$maxDistance": 30000,
            }
        }
    }
    cur = db.companies_clean.find(query, {"_id":0})
    companies_number_air.append((cur.count()))
    companies_geolocation_air.append(e["geolocation"])

/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]


In [5]:
companies_counter_air= list(zip(companies_number_air,companies_geolocation_air))
selected_airport = max(companies_counter_air)
selected_airport

(403, {'type': 'Point', 'coordinates': [-122.386354208, 37.618217208]})

In [6]:
#Filtering companies by this point
point = selected_airport[1]
query = {
    "geolocation": {
        "$near": {
        "$geometry": point,
        "$maxDistance": 30000,
        }
    }
}
cur = db.companies_clean.find(query, {"_id":0})
print(cur.count())
result = list(cur)

403


/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


In [7]:
companies_filtered = pd.DataFrame(result)

In [8]:
def easyLatLng(row):
    gl = row.geolocation
    return {
        "latitude":gl["coordinates"][1],
        "longitude":gl["coordinates"][0]
    }

In [9]:
companies_filtered = pd.concat([companies_filtered, companies_filtered.apply(easyLatLng, axis=1, result_type="expand")], axis=1)

In [10]:
companies_filtered.to_json("../output/companies_filtered_air.json",orient="records")

In [11]:
bar = list(db.bar.find({}))

In [12]:
#Searching the bar with more companies in 1 km
companies_number_bar=[]
companies_geolocation_bar=[]
for e in bar:
    query = {
        "geolocation": {
            "$near": {
            "$geometry": e["geolocation"],
            "$maxDistance": 1000,
            }
        }
    }
    cur = db.companies_filtered_air.find(query, {"_id":0})
    companies_number_bar.append((cur.count()))
    companies_geolocation_bar.append(e["geolocation"])
    
max(companies_number_bar)

/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]


5

In [13]:
companies_counter_bar = list(zip(companies_number_bar, companies_geolocation_bar))
selected_bar=[]
for e in companies_counter_bar:
    if e[0]==max(companies_number_bar):
        selected_bar.append(e[1])
        break
selected_bar

[{'type': 'Point', 'coordinates': [-122.4232475424, 37.7551760569]}]

In [14]:
#Filtering companies by this point
point3 = selected_bar[0]
query3 = {
    "geolocation": {
        "$near": {
        "$geometry": point3,
        "$maxDistance": 1000,
        }
    }
}
cur = db.companies_filtered_air.find(query3, {"_id":0})
print(cur.count())
result = list(cur)

5


/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
companies_filtered_bar = pd.DataFrame(result)
companies_filtered_bar

,name,category_code,total_money_raised,geolocation,clean_state,latitude,longitude
0,Netbiscuits,tech,27000000,"{'type': 'Point', 'coordinates': [-122.4264417...",success,37.755037,-122.426442
1,RateItAll,tech,1400000,"{'type': 'Point', 'coordinates': [-122.418137,...",success,37.755012,-122.418137
2,OQO,tech,39800000,"{'type': 'Point', 'coordinates': [-122.415869,...",success,37.759117,-122.415869
3,Pageflakes,tech,4099999,"{'type': 'Point', 'coordinates': [-122.414689,...",success,37.758113,-122.414689
4,Aardvark,tech,6000000,"{'type': 'Point', 'coordinates': [-122.415813,...",success,37.748676,-122.415813


In [16]:
companies_filtered_bar.to_json("../output/companies_filtered_bar.json",orient="records")

In [17]:
starbucks = list(db.starbucks.find({}))

In [18]:
#Searching the Starbucks with more companies in 500 m
companies_number_str=[]
companies_geolocation_str=[]
for e in starbucks:
    query = {
        "geolocation": {
            "$near": {
            "$geometry": e["geolocation"],
            "$maxDistance": 500,
            }
        }
    }
    cur = db.companies_filtered_bar.find(query, {"_id":0})
    if (cur.count())!=0:
        companies_number_str.append((cur.count()))
        companies_geolocation_str.append(e["geolocation"])

/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  del sys.path[0]
/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


In [19]:
companies_counter_str = list(zip(companies_number_str, companies_geolocation_str))
selected_starbuck=[]
for e in companies_counter_str:
    if e[0]==max(companies_number_str):
        selected_starbuck.append(e[1])
        break
selected_starbuck

[{'type': 'Point', 'coordinates': [-122.41, 37.76]}]

In [21]:
#Only one company, this is our location
point = {'type': 'Point', 'coordinates': [-122.41, 37.76]}
query = {
    "geolocation": {
        "$near": {
        "$geometry": point,
        "$maxDistance": 500,
        }
    }
}
cur = db.companies_filtered_bar.find(query, {"_id":0})
print(cur.count())
result = list(cur)

1


/Users/manuelescamezsanchez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


In [23]:
company_selected = pd.DataFrame(result)
company_selected

,name,category_code,total_money_raised,geolocation,clean_state,latitude,longitude
0,Pageflakes,tech,4099999,"{'type': 'Point', 'coordinates': [-122.414689,...",success,37.758113,-122.414689


In [24]:
company_selected.to_json("../output/company_selected.json",orient="records")